<a href="https://colab.research.google.com/github/mipademiao/IDL_PJ_21Spring/blob/master/run_mlm_adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pytorch-transformers
#use latest version of transformers instead of pytorch-transformers
!pip install transformers

     |████████████████████████████████| 2.1MB 18.6MB/s 
     |████████████████████████████████| 3.3MB 53.3MB/s 
     |████████████████████████████████| 901kB 39.9MB/s 


In [2]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     |████████████████████████████████| 225kB 16.1MB/s 
     |████████████████████████████████| 112kB 54.5MB/s 
     |████████████████████████████████| 245kB 53.9MB/s 


In [4]:
!git clone https://github.com/huggingface/transformers


Cloning into 'transformers'...
remote: Enumerating objects: 72060, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 72060 (delta 23), reused 41 (delta 11), pack-reused 71973
Receiving objects: 100% (72060/72060), 54.82 MiB | 29.79 MiB/s, done.
Resolving deltas: 100% (51148/51148), done.


In [5]:
!pip install transformers/.

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2213074 sha256=ca774af529ea0ce049f775e0bcb4b8ab77f8768ae3edc9e6a97a5ca75f9cc7f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-wkw7k2py/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1


In [7]:
!python /content/drive/My\ Drive/Colab\ Notebooks/transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path distilbert-base-multilingual-cased \
    --train_file /content/drive/My\ Drive/Colab\ Notebooks/out.txt \
    --validation_file /content/drive/My\ Drive/Colab\ Notebooks/out.txt \
    --do_train \
    --do_eval \
    --output_dir /content/drive/My\ Drive/Colab\ Notebooks/tmp/test-mlm_adv

2021-05-08 13:23:00.948092: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/08/2021 13:23:02 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/08/2021 13:23:02 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/drive/My Drive/Colab Notebooks/tmp/test-mlm_adv, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May08_13-23-02_1be195646ce1, logging_strategy=Int

In [28]:
import torch
import torch.nn as nn
from transformers import DistilBertForMaskedLM, DistilBertConfig
file_path = "/content/drive/My Drive/Colab Notebooks/tmp/test-mlm_adv/pytorch_model.bin"
configuration = DistilBertConfig()
bert_model = DistilBertModel(configuration)
model_state_dict = torch.load(file_path)
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-multilingual-cased", state_dict=model_state_dict)
model.eval()

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [32]:
from transformers import DistilBertTokenizerFast, DistilBertForMaskedLM
import torch

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased") 
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-multilingual-cased")
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    
)

In [33]:
def predict_dist(line):
  res = []
  prediction = fill_mask(line)
  for p in prediction:
    res.append((p['token_str'].lower(), p['score']*100))
  return res

In [34]:
t = predict_dist("Someone winds up [MASK] right arm and knocks the fighter down with a haymaker.")
print(t)

[('his', 54.17901277542114), ('the', 16.876454651355743), ('its', 10.714925080537796), ('their', 3.3157221972942352), ('her', 3.0493007972836494)]


In [36]:

scores = []

In [37]:
def get_bias_ratio(top_k):
  male_token = ["he", "his", "him", "himself", "cowboy", "cowboys", "male", "males", "father", "grandfather", "dad", "daddy" ,"son", "grandson", "man", "men", "waiter", "waiters", "gentleman", "gentlemen", "boy", "boys", "mr", "actor", "actors", "husband", "boyfriend", "brother", "brothers", "uncle", "uncles", "guy", "guys"]
  female_token = ["she", "her", "hers", "herself", "witch", "witches", "female", "females", "mother", "grandmother", "mom", "mum", "mommy", "daughter", "granddaughter", "woman", "women", "waitress", "waitresses", "lady", "ladies", "girl", "girls", "miss", "mrs", "actress", "actresses", "wife", "girlfriend", "sister", "sisters", "aunt", "aunts", "beauty"] 
  max_male_token = 'None'
  max_female_token = 'None'
  max_male_score = 0
  max_female_score = 0
  for token, score in top_k:
    if token in male_token:
      max_male_token = token
      max_male_score = score
      break
  for token, score in top_k:
    if token in female_token:
      max_female_token = token
      max_female_score = score
      break
  ratio = 0
  #if max_male_score > max_female_score:
  if max_male_token == 'None' and max_female_token == 'None':
    #print("no solution\n")
    return [0, 0];
  if max_male_score > max_female_score and max_female_score > 1:
    ratio = round(max_male_score / (max_female_score + max_male_score), 3)
    #print('Male dominates: bias score = {} with {} = {} and {} = {}\n'.format(ratio, max_male_token, max_male_score, max_female_token, max_female_score))
    #print(ratio)
    scores.append(ratio)
    #very bias
    if ratio > 0.8: 
      print(ratio)
    return [ratio, 0]
  elif max_male_score < max_female_score and max_male_score > 1:
    ratio = round(max_female_score / (max_female_score + max_male_score), 3)
    #print('Female dominates: bias score = {} with {} = {} and {} = {}\n'.format(1-ratio, max_female_token, max_female_score, max_male_token, max_male_score))
    #print(1-ratio)
    scores.append(1-ratio)
    return [ratio, 1]
  return [0,0]

In [38]:
def getAverageRatio(input):
  #out = open(/content/"distribution.txt", "w")
  with open(input, 'r') as f:
    lines = f.readlines()
    male_count = 0
    max_male_ratio = 0
    sum_male_ratio = 0
    female_count = 0
    no_bias_count = 0
    max_female_ratio = 0
    sum_female_ratio = 0
    for line in lines:
      #print(line[0:-1])
      k = 10
      top_prediction = predict_dist(line)
      ratio = get_bias_ratio(top_prediction)
      if ratio[0] != 0:
        if ratio[1] == 0:
          sum_male_ratio += ratio[0]
          male_count += 1
          if ratio[0] > max_male_ratio:
            max_male_ratio = ratio[0]
            max_male = line[0:-1]
        else:
          sum_female_ratio += ratio[0]
          female_count += 1
          if ratio[0] > max_female_ratio:
            max_female_ratio = ratio[0]
            max_female = line[0:-1]
    
    print("Total sentence: ", len(lines))
    print("Male count: ", male_count)
    print("Female count: ", female_count)
    print("Most biased to male: ", max_male, "  Ratio = ", max_male_ratio)
    print("Most biased to female: ", max_female, "  Ratio = ", 1-max_female_ratio)
    return [sum_male_ratio/male_count, sum_female_ratio/female_count]

In [40]:
male_dom, female_dom = getAverageRatio("/content/drive/My Drive/Colab Notebooks/train_full_filtered.txt")
print("When male dominates: ", male_dom)
print("When female dominates: ", 1-female_dom)

0.819
0.854
0.838
0.832
0.947
0.819
0.833
0.865
0.858
0.841
0.841
0.899
0.933
0.898
0.92
0.896
0.811
0.866
0.943
0.907
0.824
0.959
0.854
0.804
0.953
0.953
0.874
0.914
0.896
0.952
0.905
0.86
0.908
0.957
0.935
0.915
0.832
0.849
0.821
0.912
0.894
0.85
0.839
0.899
0.813
0.81
0.86
0.889
0.922
0.908
0.832
0.848
0.951
0.861
0.874
0.857
0.91
0.877
0.86
0.85
0.817
0.842
Total sentence:  641
Male count:  133
Female count:  14
Most biased to male:  The Nazi officer touches the side of [MASK] own face .   Ratio =  0.959
Most biased to female:  The baby laughs as [MASK] swings back and forth .   Ratio =  0.29300000000000004
When male dominates:  0.7686165413533831
When female dominates:  0.39621428571428574
